In [47]:
import random
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
from functools import partial

Wir stellen das Spielfeld als Liste von neun Strings (' ', 'X' oder 'O') dar. 

In [80]:
class Tic_Tac_Toe:
    
    def __init__(self, computer=None):
        self.board = [ ' ' ] * 9
        self.turn = 'X'
        self.finished = False
        self.computer = computer
        self.init_board()
        
    def init_board(self):
        self.text_box = widgets.Text(value = f'Am Zug: {self.turn}', layout=Layout(width='129px', height='40px'))
        self.status = widgets.Text(value = '', layout=Layout(width='129px', height='40px'))
        self.buttons = []
        for i in range(9):
            button = widgets.Button(description=self.board[i],
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click to play',
            icon='',
            layout=Layout(width='40px', height='40px'))
            self.buttons.append(button)
            button.on_click(partial(self.on_button_clicked, i))
        tic_tac_toe_board = VBox([HBox(self.buttons[:3]), HBox(self.buttons[3:6]), HBox(self.buttons[6:9])])
        display(VBox([self.text_box, self.status, tic_tac_toe_board]))
        
    def update_ui(self):
        print('update_ui: ', self.turn)
        self.text_box.value = f'Am Zug: {self.turn}'
        if self.computer == self.turn:
            self.status.value = f'Ich denke nach ...'
            for b in self.buttons:
                b.disabled = True
        else:
            for b in self.buttons:
                b.disabled = False
        
    def on_button_clicked(self, index, button):
        print(f"button clicked: {index}")
        if self.finished:
            return
        
        self.move(index, self.turn)
        if self.check_winner():
            winner = self.check_winner()
            if winner == '-':
                self.text_box.value = 'Das Spiel endet unentschieden'
            else:
                self.text_box.value = f'{winner} gewinnt'
            self.finished = True
            return
        
        if self.computer == self.turn:
            move, value = self.computer_move()
            self.move(move, self.computer)
            self.status.value = f'AI: {move} ({value})'
        
        if self.check_winner():
            winner = self.check_winner()
            if winner == '-':
                self.text_box.value = 'Das Spiel endet unentschieden'
            else:
                self.text_box.value = f'{winner} gewinnt'
            self.finished = True
            return

            
    def move(self, index, player):
        self.buttons[index].description = player
        self.board[index] = player
        self.turn = 'O' if player == 'X' else 'X'
        self.update_ui()
            
    def computer_move(self):
        best = self.best_moves(self.board, self.turn)
        return random.choice(best[0]), best[1]
    
    def check_winner(self, board=None):
        
        if not board:
            board = self.board
    
        # Prüfe Zeilen
        for i in range(3):
            if board[i*3 + 0] == board[i*3 + 1] == board[i*3 + 2] != ' ':
                return board[i*3]
    
        # Prüfe Spalten    
        for i in range(3):
            if board[i + 0] == board[i + 3] == board[i + 6] != ' ':
                return board[i]
    
        # Prüfe Diagonalen
        if board[0] == board[4] == board[8] != ' ':
            return board[0]
        if board[2] == board[4] == board[6] != ' ':
            return board[2]
    
        if len([b for b in board if b == ' ']) == 0:
            return '-'
        else:
            return None
        
    def get_moves(self, board):
        if not board:
            board = self.board
            
        return [x for x in range(9) if board[x] not in ['X', 'O']]
    
    def eval_position(self, board, zug, player):
        _board = board.copy()
        _board[zug] = player
    
        winner = self.check_winner(_board)
        if winner:
            if winner == player:
                return 1, [zug]
            elif winner == '-':
                return 0, [zug]
            else: 
                return -1, [zug]
            
        # Kein Endknoten; weitere Evaluation durch Rekursion 
        moves = self.get_moves(_board)
        opponent = 'X' if player == 'O' else 'O'
        
        # Rekursion: Evaluiere mögliche Züge des Gegners
        values = { move: self.eval_position(_board, move, opponent) for move in moves }

        # Der Gegner wird den für sich besten Zug auswählen.
        # Wir teilen den "Wert" des Zuges durch die Länge der Zugfolge, damit ein kürzerer
        # Gewinnweg bevorzugt wird.
        (move, result) = max(values.items(), key= lambda p: p[1][0] / len(p[1][1]))
        # Ein Gewinn für den Gegner ist ein Verlust für und – daher müssen wir das Vorzeichen wechseln
        return -result[0], [zug] + result[1]
            
    def best_moves(self, board, player): 
        # Bewerte den Wert aller möglichen Züge
        results = { move: self.eval_position(board, move, player)[0] for move in self.get_moves(board) }
    
        # Wähle die besten Züge aus
        best_result = max(results.values())
        best_moves = [ move for move in results.keys() if results[move] == best_result ]
    
        return best_moves, best_result

In [81]:
ttt = Tic_Tac_Toe(computer='O')